In [18]:
import requests
from pprint import pprint
import pandas as pd
import zipfile
import io
import json
import os
os.makedirs("results/inputs_extracted_xml", exist_ok=True)
year = "2022"

# Source = "https://www.prix-carburants.gouv.fr/rubrique/opendata/" (site gouvernemental)
url = f"https://donnees.roulez-eco.fr/opendata/annee/{year}"

response = requests.get(url)
print(response)

zip_file = zipfile.ZipFile(io.BytesIO(response.content))
file_name = zip_file.namelist()[0]
print(file_name)

with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
    zip_ref.extractall("results/inputs_extracted_xml")
print(file_name, "extracted")

<Response [200]>
PrixCarburants_annuel_2022.xml
PrixCarburants_annuel_2022.xml extracted


In [19]:
import xml.etree.ElementTree as ET
import html

tree = ET.parse(f"results/inputs_extracted_xml/PrixCarburants_annuel_{year}.xml")
root = tree.getroot()

data = []
for pdv in root.findall("pdv"):
    for p in pdv.findall("prix"):
        row = {
            "id": pdv.get("id"),
            "latitude": pdv.get("latitude"),
            "longitude": pdv.get("longitude"),
            "cp": pdv.get("cp"),
            "ville": pdv.find("ville").text,
            # "ville": (pdv.find("ville").text.encode("latin1").decode("utf-8") if pdv.find("ville") is not None and pdv.find("ville").text else None),
            # "adresse": html.unescape(pdv.find("adresse").text).replace(",", " ").replace(";", " ") if pdv.find("adresse") is not None and pdv.find("adresse").text else None,
            "adresse": pdv.find("adresse").text.replace(",", " ").replace(";", " ") if pdv.find("adresse") is not None and pdv.find("adresse").text else None,
            "nom": p.get("nom"),
            "maj": p.get("maj").split("T")[0] if p.get("maj") is not None and "T" in p.get("maj") else None,
            "valeur": float(p.get("valeur")) if p.get("valeur") is not None else None
        }
        data.append(row)

df = pd.DataFrame(data)

os.makedirs("results/inputs_csv", exist_ok=True)
df.to_csv(f"results/inputs_csv/PrixCarburants_annuel_{year}.csv", index=False)
df.head(25)

Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
  File "C:\Users\GaetanCORIN\anaconda3\envs\python311\Lib\site-packages\ipykernel\iostream.py", line 684, in write
    self._buffers[frozenset(parent.items())].write(string)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\GaetanCORIN\anaconda3\envs\python311\Lib\site-packages\dateutil\tz\tz.py", line 74, in utcoffset
    def utcoffset(self, dt):

KeyboardInterrupt: 


KeyboardInterrupt: 

In [10]:
print(df[df["id"]== "13140008"])

              id latitude longitude     cp    ville  \
439603  13140008  4358515    500513  13140  Miramas   

                          adresse   nom   maj  valeur  
439603  RN 569 Route d#039 Istres  None  None     NaN  


In [15]:
# print(df["valeur"].min(), df["valeur"].max())

In [17]:
len(df["id"].unique())

13814